In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import pandas as pd

pd.set_option('display.max_columns', 100)

from modules import losses, models, samplers, searches, regularizers, evaluators, trainers, datasets, distributions

# ML100k

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
print(f"n_user={n_user}, n_item={n_item}")
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

n_user=940, n_item=1447


In [3]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"       : evaluators.recall,
    "Unpopularity" : evaluators.unpopularity,
    "Unpopularity2": evaluators.unpopularity2,
    "Unpopularity3": evaluators.unpopularity3,
    "F1-score"     : evaluators.f1_score,
    "F1-score2"    : evaluators.f1_score2,
    "F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [4]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [5]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [6]:
trainer.fit(n_batch, n_epoch)

In [7]:
print('knn')
trainer.valid(knn, userwise)
display(trainer.valid_scores)

knn


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.398098,0.877321,3.318121,0.014806,0.491728,0.681867,0.025782,0.574217,0.891321,3.635276,0.050278,0.652257,0.968253,0.083014,0.889332,0.887826,4.74945,0.437967,0.872784,1.489182,0.513129


In [8]:
print('mp')
trainer.valid(mp, userwise)
display(trainer.valid_scores)

mp


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.320669,0.925621,4.275788,0.035536,0.421298,0.570852,0.054792,0.487213,0.931345,4.55206,0.084522,0.584765,0.854416,0.130203,0.850493,0.899975,5.212485,0.473324,0.850957,1.448525,0.554126


# ML200m
ランダムに100kサンプリング

In [9]:
dataset = datasets.ML20mTo100k()
n_user = dataset.n_user
n_item = dataset.n_item
print(f"n_user={n_user}, n_item={n_item}")
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

n_user=1371, n_item=2974


In [10]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"       : evaluators.recall,
    "Unpopularity" : evaluators.unpopularity,
    "Unpopularity2": evaluators.unpopularity2,
    "Unpopularity3": evaluators.unpopularity3,
    "F1-score"     : evaluators.f1_score,
    "F1-score2"    : evaluators.f1_score2,
    "F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [11]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [12]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [13]:
trainer.fit(n_batch, n_epoch)

In [14]:
print('knn')
trainer.valid(knn, userwise)
display(trainer.valid_scores)

knn


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.387977,0.999068,10.55137,0.337375,0.415659,0.715,0.216151,0.720824,0.999139,10.646955,0.704452,0.757243,1.32679,0.651858,1.0,0.319635,3.421265,0.999915,0.471238,1.509032,0.999955


In [15]:
print('mp')
trainer.valid(mp, userwise)
display(trainer.valid_scores)

mp


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.379711,0.999117,10.61364,0.346623,0.406044,0.699928,0.215316,0.712157,0.999156,10.667319,0.708936,0.749131,1.311198,0.645854,1.0,0.319635,3.421265,0.999915,0.471238,1.509032,0.999955


# ML200m
ユーザ数を1000にするようにサンプリング

In [16]:
dataset = datasets.ML200mTo100kByUser()
n_user = dataset.n_user
n_item = dataset.n_item
print(f"n_user={n_user}, n_item={n_item}")
train_set, test_set = dataset.get_train_and_test_set()

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

/home/taichi-nakashima/project/mpcml/experiment/test/../../modules/datasets/BaseDataset.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rating["rating"] = (df_rating["rating"] >= 4.0).astype(int)


n_user=995, n_item=6148


In [17]:
# k
ks = [5, 10, 50]

score_function_dict = {
    "Recall"       : evaluators.recall,
    "Unpopularity" : evaluators.unpopularity,
    "Unpopularity2": evaluators.unpopularity2,
    "Unpopularity3": evaluators.unpopularity3,
    "F1-score"     : evaluators.f1_score,
    "F1-score2"    : evaluators.f1_score2,
    "F1-score3"    : evaluators.f1_score3
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)
# coverage = evaluators.CoverageEvaluator(test_set, ks)
# hubness = evaluators.HubnessEvaluator(test_set, ks)

In [18]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [19]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 50
valid_per_epoch = 10
n_item_sample = 30
n_user_sample = 30
no_progressbar = True

# models
model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)

# distributiuons
gaussian = distributions.Gaussian()
gamma = distributions.Gamma()

# search
knn = searches.NearestNeighborhood(model)
mp = searches.MutualProximity(model, gamma)

# learning late optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [20]:
trainer.fit(n_batch, n_epoch, knn, userwise, valid_per_epoch)

In [21]:
print('knn')
trainer.valid(knn, userwise)
display(trainer.valid_scores)

knn


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.399427,0.981303,6.27172,0.009479,0.506492,0.727673,0.013546,0.573383,0.984145,6.732725,0.040357,0.671342,1.036375,0.060814,0.87222,0.92874,8.10471,0.433602,0.873317,1.563715,0.497408


In [22]:
print('mp')
trainer.valid(mp, userwise)
display(trainer.valid_scores)

mp


,Recall@5,Unpopularity@5,Unpopularity2@5,Unpopularity3@5,F1-score@5,F1-score2@5,F1-score3@5,Recall@10,Unpopularity@10,Unpopularity2@10,Unpopularity3@10,F1-score@10,F1-score2@10,F1-score3@10,Recall@50,Unpopularity@50,Unpopularity2@50,Unpopularity3@50,F1-score@50,F1-score2@50,F1-score3@50
0,0.294365,0.991112,7.765634,0.028408,0.393002,0.5476,0.034638,0.446607,0.991967,8.115132,0.075512,0.552466,0.824885,0.103898,0.80659,0.930835,8.742991,0.472113,0.821775,1.457482,0.542009
